# Introduction

A message is coded by changing letters according to the following algorithm.  Number the letters from 0 to 25 and add $a x + b$ to each letter.  $a$ and $b$ are positive integers, as is $x$.  However, $x$ is incremented by one for each character in the message.  The resulting number is reduced to the interval $[0, 25]$ by substracting 26 repeatedly.

Decoding the message is similar, bu $a x + b$ is substracted from the letter's number and 26 is added repeatedly until the resulting number represents a letter.

To decode a coded message, you need the values $a$ and $b$, so they form the encryption key.

# Encoding/decoding characters

## Test for correctness

To verify this implementation, we can encode and decode the lowercase letters for a range of values of $a$, $b$ and $x$, and check that the original character is recovered when it is decoded.

In [1]:
import string

In [2]:
def test_char_encryption(encrypt, decrypt, max_x=100):
    for char in string.ascii_lowercase:
        for a in range(20):
            for b in range(20):
                for x in range(max_x):
                    coded = encrypt(char, a, b, x)
                    if not coded.islower():
                        print(f'problem for {char} -> {coded} with {a}, {b}, {x}')
                    decoded = decrypt(coded, a, b, x)
                    if decoded != char:
                        print(f'problem for {char} -> {coded} -> {decoded} using {a}, {b}, {x}')

## Simple solution

Simple implementation to encode and decode a single character could be the following.

In [3]:
def encode_simple(char, a, b, x):
    new_char_ascii = ord(char) + a*x + b
    while new_char_ascii > ord('z'):
        new_char_ascii -= 26
    return chr(new_char_ascii)

In [4]:
encode_simple('a', 5, 7, 3)

'w'

In [5]:
def decode_simple(char, a, b, x):
    new_char_ascii = ord(char) - (a*x + b)
    while new_char_ascii < ord('a'):
        new_char_ascii += 26
    return chr(new_char_ascii)

In [6]:
decode_simple('w', 5, 7, 3)

'a'

Run the test and time.

In [7]:
%time test_char_encryption(encode_simple, decode_simple)

CPU times: user 5.38 s, sys: 46 ms, total: 5.42 s
Wall time: 5.59 s


A drawback of this approach is that many iterations of the while loop are required for long messages, as $x$ increases linearly with the length of the message.

## Implementation using modulo operator

The modulo operator can be used to simplify the implementation and eliminate the issue that is inherent in the starightforward implementation.

In [8]:
def encode_char(char, a, b, x):
    delta = a*x + b
    return chr(ord('a') + (ord(char) - ord('a') + delta) % 26)

In [9]:
encode_char('a', 5, 7, 3)

'w'

In [10]:
def decode_char(char, a, b, x):
    delta = a*x + b
    return chr(ord('a') + (ord(char) - ord('a') - delta) % 26)

In [11]:
decode_char('w', 5, 7, 3)

'a'

Run the test and time.

In [12]:
%time test_char_encryption(encode_char, decode_char)

CPU times: user 1.23 s, sys: 2.84 ms, total: 1.23 s
Wall time: 1.26 s


Note that this implementation is significantly faster for texts that are less than 100 characters long.

# Encoding/decoding texts

## Test for encryption/decryption

Test function to encrypt/decrypt text.

In [13]:
import random

In [14]:
def random_text(nr_chars):
    return ''.join(random.choices(string.ascii_lowercase, k=nr_chars))

In [15]:
def test_text_encryption(crypt_function, nr_chars=10_000):
    text = random_text(nr_chars)
    for a in range(1, 20):
        for b in range(1, 20):
            encoding_text = crypt_function(text, a, b, True)
            decoding_text = crypt_function(encoding_text, a, b, False)
            if  text != decoding_text:
                print(f'problem for {a} and {b}')

## Crypt function

The `crypt` function takes a text to encryp/dectypt and the $a$ and $b$>

In [16]:
def crypt(text, a, b, is_encode):
    crypt_function = encode_char if is_encode else decode_char
    x = 0
    new_text = ''
    for char in text:
        if char.islower():
            char = crypt_function(char, a, b, x)
            x += 1
        new_text += char
    return new_text

In [20]:
encoded = crypt('this is a test', 5, 7, True)
print(encoded)

atzo jy l jzsy


In [21]:
crypt(encoded, 5, 7, False)

'this is a test'

Run the test and time.

In [19]:
test_text_encryption(crypt)